In [1]:
""" This dashboard uses a custom mongo_crud.py module to show an interactive data table with an
    unfiltered view of the Austin Animal Center Outcomes dataset and a corresponding geolocation chart
    to display location, name, and breed of the selected row in the data table.
    Users can click through pages of 10 documents at a time or go straight to the end documents.
    Users can sort the table in ascending or descending order of any column.
    Users can limit number of displayed documents at any given time.
    Users can display only dogs, only cats, or all animals at once.
"""
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
from dash import callback_context

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import datetime

from mongo_crud import MongoCrud


###########################
# Data Manipulation / Model
###########################

# Database connection instance
username = "aacuser"
password = "M22"
host = 'nv-desktop-services.apporto.com'
port = 32474
database = 'AAC'
collection = 'animals'

db = MongoCrud(username, password, host, port, database, collection)

df = pd.DataFrame.from_records(db.read({}))  # JSON input & returned list object (empty object returns all documents)

# returns a new dataframe that does not contain the dropped column(s) to avoid invalid _id (ObjectID) type error
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Austin Animal Center: Dashboard'))),
    html.Hr(),
    
    html.Div(className='buttonRow',
            style={'display' : 'flex'},
                children=[  # NOTE: switch to Dash States if scaling
                    html.Button(id='cats-button', n_clicks=0, children='Cats'),
                    html.Button(id='dogs-button', n_clicks=0, children='Dogs'),
                    html.Button(id='all-button', n_clicks=0, children='All')
                ]
            ),
    
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True}
            for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",  # enables built-in pagination for table UX
        page_current=0,  # current page number
        page_size=10,    # number of rows to display on a single page
    ), 
    
    # input box to limit rows
    dcc.Input(
        id='limit-input',
        type='number',
    ),
    
    # Map constructor
    html.Br(),  # line break
    html.Hr(),  # horizontal line
    dl.Map(
        id='map-id',
        children=[
            dl.TileLayer(),
            dl.LayerGroup(id='marker-layer')  # Container to dynamicaally add markers
        ],
        className='col s12 m6',  # Google Material Design specification
     ),
    
    # unique tag displayed after other layout elements
    html.H4("Matthew Pool: MongoDB-Dash-Pandas (MVC)", id="m.pool")
])

#############################################
# Interaction Between Components / Controller
#############################################
@app.callback(
    Output('datatable-id', 'page_size'),
    [Input('limit-input', 'value')],
    [State('datatable-id', 'page_size')]
)
def update_page_size(input_value, current_page_size):
    if input_value is not None and input_value > 0:  # Check if input is positive
        return input_value
    else:
        return current_page_size  # if invalid input value
    
@app.callback(Output('datatable-id', 'data'),
             [Input('cats-button', 'n_clicks'),
              Input('dogs-button', 'n_clicks'),
              Input('all-button', 'n_clicks')
             ])
def on_click(cats_button, dogs_button, all_button):  # parameters in same order as callback inputs
    # start case
    df = pd.DataFrame.from_records(db.read({}))
    
    context = callback_context
    button_id = context.triggered[0]['prop_id'].split('.')[0] if context.triggered else all_button
    
    if button_id == 'cats-button':
        df = pd.DataFrame.from_records(db.read({'animal_type': 'Cat'}))
    elif button_id == 'dogs-button':
        df = pd.DataFrame.from_records(db.read({'animal_type': 'Dog'}))
    elif button_id == 'all-button':
        df = pd.DataFrame.from_records(db.read({}))  # all documents
            
    # Cleanup Mongo _id field
    df.drop(columns=['_id'],inplace=True)
    return df.to_dict('records')

#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows')]
)
def update_styles(selected_rows):
    if not selected_rows:
        return []  # return empty style condition
    
    return [{
        'if': { 'row_index': i },  # stylize rows (changed from column_id)
        'backgroundColor': '#D2F3FF'  # changed from background_color
    } for i in selected_rows or []]

# Callback to update geo-location chart for the selected data entry
# derived_virtual_data will be the set of (dict) data available from the datatable
# derived_virtual_selected_rows will be the (list) selected row(s) in the table
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    if not viewData or not index:
        return []  # don't update markers if no data or rows are selected

    """
    # Checks if selected_rows is empty to prevent IndexError
    row_index = selected_rows[0] if selected_rows else 0
    
    # Prevents IndexError if row_index is out of range of ViewData
    if row_index >= len(dff):
        return[]
    """
    
    # Geolocation chart  
    dff = pd.DataFrame.from_dict(viewData)
    # List can be converted to a row index here since using single-row selection
    if index is None:
        row = 0
    else: 
        row = index[0]  # Austin TX is at [30.75,-97.48]
    
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[30.75,-97.48],
               zoom=9,
               children=[
                   dl.TileLayer(id="base-layer-id"),
                   # Marker with tool tip and popup
                   # location_lat=col14, location_long=col15, breed=col5, name=col10
                   dl.Marker(position=[dff.iloc[row, 13],dff.iloc[row,14]],
                             children=[
                                 dl.Tooltip(dff.iloc[row,4]),
                                 dl.Popup([
                                     html.H3("Animal Name"),
                                     html.P(dff.iloc[row,9])
                                 ])
                             ]
                            )
               ]
              )
    ]

# Dash app running on http://127.0.0.1:15185/
app.run_server(debug=True)

INFO:root:Program Executing...


Dash app running on http://127.0.0.1:15185/
